<a href="https://colab.research.google.com/github/AbdulAhadSiddiqui-0786/Voice-AI/blob/main/Voice_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytube pydub openai-whisper pyannote.audio torch torchvision torchaudio transformers -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.8/897.8 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.5/828.5 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 41.4 MB/s

In [ ]:
!pip install yt-dlp -q
from pydub import AudioSegment

url = "https://www.youtube.com/watch?v=4ostqJD3Psc"

# Download audio with yt-dlp
!yt-dlp -x --audio-format mp3 -o "call.%(ext)s" {url}

# Convert to wav, mono, 16kHz
audio = AudioSegment.from_file("call.mp3")
audio = audio.set_channels(1).set_frame_rate(16000)
audio.export("call.wav", format="wav")

print(" Audio downloaded & converted to WAV using yt-dlp!")


[youtube] Extracting URL: https://www.youtube.com/watch?v=4ostqJD3Psc
[youtube] 4ostqJD3Psc: Downloading webpage
[youtube] 4ostqJD3Psc: Downloading tv simply player API JSON
[youtube] 4ostqJD3Psc: Downloading tv client config
[youtube] 4ostqJD3Psc: Downloading tv player API JSON
ERROR: [youtube] 4ostqJD3Psc: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
 Audio downloaded & converted to WAV using yt-dlp!


In [ ]:
from pyannote.audio import Pipeline
from google.colab import userdata
from tqdm import tqdm
import soundfile as sf
from pydub import AudioSegment
import math
import os
import pickle

#  Hugging Face token
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

# 1. Load diarization model
print(" Loading diarization model...")
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization",
    use_auth_token=HUGGINGFACE_TOKEN
)
print(" Model loaded!")

# 2. Load audio and get duration
audio_file = "call.wav"
info = sf.info(audio_file)
duration = info.duration  # in seconds
print(f" Audio duration: {duration:.1f} sec")

# 3. Split into 30s chunks
chunk_size = 30_000  # in ms
audio = AudioSegment.from_wav(audio_file)
chunks = math.ceil(len(audio) / chunk_size)
print(f" Splitting into {chunks} chunks of 30s each...")

# 4. Process chunks with caching
speaker_segments = []
with tqdm(total=chunks, desc="Processing chunks") as pbar:
    for i in range(chunks):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, len(audio))
        chunk_file = f"chunk_{i}.wav"
        cache_file = f"chunk_{i}_diarization.pkl"

        # Export chunk only if not exists
        if not os.path.exists(chunk_file):
            audio[start:end].export(chunk_file, format="wav")

        # Load from cache if exists
        if os.path.exists(cache_file):
            with open(cache_file, "rb") as f:
                diarization = pickle.load(f)
        else:
            # Run diarization on this chunk
            diarization = pipeline(chunk_file)
            with open(cache_file, "wb") as f:
                pickle.dump(diarization, f)

        # Extract speaker segments
        for turn, _, speaker in diarization.itertracks(yield_label=True):
            speaker_segments.append({
                "speaker": speaker,
                "start": turn.start + start / 1000,  # shift time
                "end": turn.end + start / 1000,
                "duration": turn.end - turn.start
            })

        pbar.update(1)

print(" Diarization complete!")
print(" Speakers found:", set([s["speaker"] for s in speaker_segments]))


 Loading diarization model...


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`
/usr/local/lib/python3.12/dist-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Using symlink found at '/root/.cache/torch/pyannote/sp

Model was trained with pyannote.audio 0.0.1, yours is 3.4.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.8.0+cu126. Bad things might happen unless you revert torch to 1.x.


DEBUG:speechbrain.utils.parameter_transfer:Collecting files (or symlinks) for pretraining in /root/.cache/torch/pyannote/speechbrain.
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/embedding_model.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /root/.cache/torch/pyannote/speechbrain/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/classifier.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["classifier"] = /root/.cache/torch/pyannote/speech

 Model loaded!
 Audio duration: 122.7 sec
 Splitting into 5 chunks of 30s each...


Processing chunks: 100%|██████████| 5/5 [00:00<00:00, 3430.08it/s]

 Diarization complete!
 Speakers found: {'SPEAKER_03', 'SPEAKER_01', 'SPEAKER_00', 'SPEAKER_02', 'SPEAKER_04'}


In [ ]:
import whisper

# Load Whisper model
model = whisper.load_model("small")   # small model = good balance (fast + accurate)

# Transcribe audio
result = model.transcribe("call.wav")
transcript = result["text"]

print(" Transcription complete!")
print("Sample transcript:", transcript[:250], "...")


100%|███████████████████████████████████████| 461M/461M [00:17<00:00, 28.0MiB/s]
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Transcription complete!
Sample transcript:  Thank you for calling Nissan. My name is Lauren. Can I have your name? Yeah, my name is John Smith. Thank you, John. How can I help you? I was just calling about to see how much it would cost to update the map in my car. I'd be happy to help you wit ...


In [ ]:
from transformers import pipeline
import re

# 1. Talk-time ratio
talk_time = {}
for seg in speaker_segments:
    talk_time[seg["speaker"]] = talk_time.get(seg["speaker"], 0) + seg["duration"]

total_time = sum(talk_time.values())
talk_ratios = {sp: round((dur/total_time)*100, 2) for sp, dur in talk_time.items()}

# 2. Number of questions
num_questions = transcript.count("?")
extra_questions = len(re.findall(r"\b(what|why|how|when|where|can|do|is|are|does|did)\b", transcript.lower()))
question_count = max(num_questions, extra_questions)

# 3. Longest monologue
longest_monologue = max([seg["duration"] for seg in speaker_segments])

# 4. Sentiment
sentiment_analyzer = pipeline("sentiment-analysis")
sentiment = sentiment_analyzer(transcript[:500])[0]  # analyze first 500 chars

# 5. Actionable insight
insight = ""
if max(talk_ratios.values()) > 70:
    insight = "One speaker dominated the conversation. Allow more balanced talk-time."
elif question_count < 3:
    insight = "Too few questions were asked. Ask more questions to engage the customer."
else:
    insight = "Good balance, but could improve listening."

# Identify roles
sales_rep = max(talk_ratios, key=talk_ratios.get)
customer = min(talk_ratios, key=talk_ratios.get)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
print("\n===== CALL QUALITY REPORT =====")
print("Talk-time ratio:", talk_ratios)
print("Questions asked:", question_count)
print("Longest monologue (sec):", round(longest_monologue, 2))
print("Call sentiment:", sentiment["label"], "| Confidence:", round(sentiment["score"], 2))
print("Actionable Insight:", insight)
print("Likely Sales Rep:", sales_rep, "| Likely Customer:", customer)



===== CALL QUALITY REPORT =====
Talk-time ratio: {'SPEAKER_00': 33.52, 'SPEAKER_01': 32.02, 'SPEAKER_02': 23.03, 'SPEAKER_03': 10.52, 'SPEAKER_04': 0.91}
Questions asked: 20
Longest monologue (sec): 11.61
Call sentiment: POSITIVE | Confidence: 1.0
Actionable Insight: Good balance, but could improve listening.
Likely Sales Rep: SPEAKER_00 | Likely Customer: SPEAKER_04
